# Import Libraries

In [ ]:
# Import the necessary libraries
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator



In [ ]:
# Define the input shape and number of classes
img_shape = (224, 224, 3)
num_classes = 9



# VGG16

In [ ]:
# Load the pre-trained VGG16 model without the top layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=img_shape)

# Freeze the weights of the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add new layers on top for classification
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dense(num_classes, activation='softmax')(x)

# Create the new model with VGG16 as the base model and the new layers on top
model = Model(inputs=base_model.input, outputs=x)

# Compile the model with categorical cross-entropy loss and Adam optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



# Import Dataset

In [ ]:
# Set the batch size and number of epochs
batch_size = 32
epochs = 10

# Create the image data generator for data augmentation and preprocessing
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load the data into the image data generator
train_generator = train_datagen.flow_from_directory('train/', target_size=img_shape[:2], batch_size=batch_size, class_mode='categorical')
validation_generator = test_datagen.flow_from_directory('validation/', target_size=img_shape[:2], batch_size=batch_size, class_mode='categorical')
test_generator = test_datagen.flow_from_directory('test/', target_size=img_shape[:2], batch_size=batch_size, class_mode='categorical')



# Model Training

In [ ]:
# Train the model with the image data generator
history = model.fit(train_generator, steps_per_epoch=len(train_generator), epochs=epochs, validation_data=validation_generator, validation_steps=len(validation_generator))

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator, steps=len(test_generator))
print('Test accuracy:', test_acc)